In [48]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

## memory 공유 확인 위한 dummy chain 생성

In [49]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

dummy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [50]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [51]:
dummy_chain = dummy_prompt | model 

In [52]:
from langchain_core.runnables.history import RunnableWithMessageHistory

dumy_chain_with_memory = RunnableWithMessageHistory(
    dummy_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [6]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 취소하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 9fe8ef79-7c3a-4121-988e-6e74f7141827 not found for run 64f9517c-4bac-4f6d-bb94-9b792ccdea52. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a307cb63-35b7-475e-9758-cafdad36f4e6-0')

In [53]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 변경하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 6c7f6fe6-4ae3-454a-9708-bf4a34e735dc not found for run a603b1b6-0a79-45b5-bff6-2015abcda523. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-59176f66-ab4a-417f-a2c1-41eb49820035-0')

## order_id 받는 부분 구현

In [7]:
chat_history = store["test_240516-1"].messages
chat_history 

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a307cb63-35b7-475e-9758-cafdad36f4e6-0')]

In [55]:
chat_history = store["test_240516-1"].messages
chat_history 

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-59176f66-ab4a-417f-a2c1-41eb49820035-0')]

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
order_change_cancel_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [10]:
classify_change_or_cancel_chain = order_change_cancel_prompt | model
classify_change_or_cancel_chain

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F54878D660>, asy

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

classify_confirmation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.
            
            To make this determination, the following conditions must be met:
            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.
            2. The user must have explicitly expressed consent in response to the AIMessage asking for approval of the action specified in action_type.
            
            Your response must be either 'yes' or 'no'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}\naction_type: {action_type}"),
    ]
)
classify_confirmation_prompt

ChatPromptTemplate(input_variables=['action_type', 'chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.\n            2. The user m

In [12]:
classify_confirmation_chain = classify_confirmation_prompt | model 
classify_confirmation_chain

ChatPromptTemplate(input_variables=['action_type', 'chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.\n            2. The user m

In [22]:
def helper(dict):
    inputs = dict["inputs"]
    action_type =  dict["action_type"]
    # action_type = action_type.content
    inputs["action_type"] = action_type
    return inputs

In [23]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

exp_chain = (
    {
        "inputs": RunnablePassthrough(),
        "action_type": classify_change_or_cancel_chain
    } | RunnableLambda(helper) | RunnablePassthrough.assign(execution_confirmation=classify_confirmation_chain)
    )
exp_chain

{
  inputs: RunnablePassthrough(),
  action_type: ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
               | ChatOpenAI(client=<openai.resource

In [24]:
response = exp_chain.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
)
response

{'chat_history': [HumanMessage(content='주문 취소하고 싶어'),
  AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a307cb63-35b7-475e-9758-cafdad36f4e6-0')],
 'input': None,
 'order_id': 7,
 'action_type': AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 87, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8f8e0b68-dd37-4260-8f06-0476ee3095de-0'),
 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 273, 'total_tokens': 274}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3a3b487d-d358-4f9b-b3c4-95cc932dcf86-0')}

### 요청 승인 여부에 따른 라우트
* 승인O -> 처리 진행
* 승인X -> 메시지 작성

In [26]:
def fetch_recent_orders(dict):
    print(dict)
    return "produdct: 백설기 quantity: 2 price: 13,000"

근데, fetch_recent_orders은 execution_confirmation이 no인 경우에만 실행되기에 exp_chain이 아니라 generate_confirm_message_chain과 결합해야

In [27]:
exp_chain2 = exp_chain | RunnablePassthrough.assign(queried_result=fetch_recent_orders)
exp_chain2 


{
  inputs: RunnablePassthrough(),
  action_type: ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
               | ChatOpenAI(client=<openai.resource

In [28]:
response = exp_chain2.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
)
response

{'chat_history': [HumanMessage(content='주문 취소하고 싶어'), AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a307cb63-35b7-475e-9758-cafdad36f4e6-0')], 'input': None, 'order_id': 7, 'action_type': AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 87, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4983409c-6a0f-4d52-be4e-eb339b2d6c6f-0'), 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 276, 'total_tokens': 277}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6d5090aa-0661-4911-a306-06134225f3c4-0')}


{'chat_history': [HumanMessage(content='주문 취소하고 싶어'),
  AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a307cb63-35b7-475e-9758-cafdad36f4e6-0')],
 'input': None,
 'order_id': 7,
 'action_type': AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 87, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4983409c-6a0f-4d52-be4e-eb339b2d6c6f-0'),
 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 276, 'total_tokens': 277}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6d5090aa-0661-4911-a306-06134225f3c4-0'),
 'queried_result': 'produdc

### 승인 메시지 작성 체인

In [36]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

generate_confirm_message_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that generates a confirmation request message based on the action_type and queried_result.
            create a message to show the queried_result and ask for final confirmation considering action_type.
            The response should be generated in Korean.
            """
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "action_type:{action_type}\nqueried_result:{queried_result}")
    ]
)

In [37]:
generate_confirm_message_chain = generate_confirm_message_prompt | model
generate_confirm_message_chain

ChatPromptTemplate(input_variables=['action_type', 'queried_result'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            You are a robot that generates a confirmation request message based on the action_type and queried_result.\n            create a message to show the queried_result and ask for final confirmation considering action_type.\n            The response should be generated in Korean.\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['action_type', 'queried_result'], template='action_type:{action_type}\nqueried_result:{queried_result}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F54878D660>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001F54878F250>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [44]:
def execution_or_message_route(info):
    print("execution_or_message_route 함수로 전달된 데이터 -> ", info)
    if "yes" in info["execution_confirmation"].content.lower():
        return "주문 변경 또는 취소 진행"
    else:
        return RunnablePassthrough.assign(queried_result=fetch_recent_orders) | generate_confirm_message_chain

In [39]:
execution_chain = exp_chain | execution_or_message_route
execution_chain

{
  inputs: RunnablePassthrough(),
  action_type: ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
               | ChatOpenAI(client=<openai.resource

이 메시지가 chat_history에 저장돼야 하는데 지금 Runnable 뭐시기에선 이게 내 생각대로 잘 구현되지 않음

In [40]:
response = execution_chain.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
)
response

execution_route 함수로 전달된 데이터 ->  {'chat_history': [HumanMessage(content='주문 취소하고 싶어'), AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a307cb63-35b7-475e-9758-cafdad36f4e6-0')], 'input': None, 'order_id': 7, 'action_type': AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 87, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-440ccfb9-1ca8-44fc-a714-0b7170496e31-0'), 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 271, 'total_tokens': 272}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4602ba6b-dd1b-48a0-9609-5e95422d1a4d-0')}
{

AIMessage(content='주문 내역 확인 요청:  \n상품: 백설기  \n수량: 2개  \n가격: 13,000원  \n\n위 주문 내역을 확인하고 주문을 취소하시겠습니까?', response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 185, 'total_tokens': 250}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4f96778b-558f-4377-ab82-5765bc1c4490-0')

In [41]:
generate_confirm_message_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that generates a confirmation request message based on the action_type and queried_result.
            Create a message to show the queried_result and ask for final confirmation considering action_type.
            If action_type is '주문 변경', instead of asking for final confirmation, include a request for the user to specify how they want to change the order if the queried order is correct.
            The response should be generated in Korean.
            """
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "action_type:{action_type}\nqueried_result:{queried_result}")
    ]
)


In [42]:
generate_confirm_message_chain = generate_confirm_message_prompt | model
generate_confirm_message_chain

ChatPromptTemplate(input_variables=['action_type', 'queried_result'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that generates a confirmation request message based on the action_type and queried_result.\n            Create a message to show the queried_result and ask for final confirmation considering action_type.\n            If action_type is '주문 변경', instead of asking for final confirmation, include a request for the user to specify how they want to change the order if the queried order is correct.\n            The response should be generated in Korean.\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['action_type', 'queried_result'], template='action_type:{action_type}\nqueried_result:{queried_result}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F54878D660>, async_client=<openai.resources.chat.completions.AsyncCompletions obj

In [45]:
execution_chain = exp_chain | execution_or_message_route
execution_chain

{
  inputs: RunnablePassthrough(),
  action_type: ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
               | ChatOpenAI(client=<openai.resource

In [56]:
response = execution_chain.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
)
response

execution_or_message_route 함수로 전달된 데이터 ->  {'chat_history': [HumanMessage(content='주문 변경하고 싶어'), AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-59176f66-ab4a-417f-a2c1-41eb49820035-0')], 'input': None, 'order_id': 7, 'action_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 85, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-30afe04b-a88f-47c3-8f5f-9db544adf1c2-0'), 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 270, 'total_tokens': 271}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9d50b9a7-6cce-4276-a7d9-fda8ffe8

AIMessage(content='주문 내역을 확인해주세요:\n상품: 백설기\n수량: 2개\n가격: 13,000원\n\n주문을 변경하시려면 변경 사항을 알려주세요. 변경하지 않으시려면 확인해 주세요.', response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 225, 'total_tokens': 297}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-76ef5c81-0069-4f34-be10-ceb67c76b88a-0')

### 승인 확인 후 작업 진행 체인

In [ ]:
def cancel_order(dict):
    order_id = dict["order_id"]
    return f"주문{order_id} 취소"

In [ ]:
def execution_route(info):
    print("execution_route 함수로 전달된 데이터 -> ", info)
    if "주문 변경" in info["action_type"].content:
        return 
    else:
        return cancel_order

In [ ]:
def cancel_order(data: dict) -> dict:
    order_id = data["order_id"]